In [1]:
import numpy as np
import pandas as pd
import re
import itertools
import copy
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
import collections
from collections import defaultdict

## Расстояние 

In [15]:
def lev_distance(a, b):
    
    # Первыми - строчки 
    # столбики - слово b
    
    global dist_matrix
    
    
    # dis = np.zeros((len(a) + 1, len(b) + 1))
    dis = [[0] * (len(b) + 1) for _ in range(len(a) + 1)] 
    i = 0
    row = 0
    col = 0

    while i < dis.size:

        if row == 0:
            if col != 0:
                dis[row, col] = dis[row, col-1] + 1

        elif col == 0:
            if row != 0:
                dis[row, col] = dis[row - 1, col] + 1
        
  
        elif row > 1 and col > 1 and a[row-1] == b[col-2] and a[row-2] == b[col-1]:
            dis[row, col] = dis[row - 3][col - 3] + 1
            
        
        else:
            dis[row, col] = min([dis[row, col - 1] + 1,  # левый 
                                    dis[row - 1, col - 1] + dist_matrix[a[row-1]][b[col-1]], # диаг               
                                    dis[row - 1, col] + 1]) # верхний
        
        col += 1
        i += 1  
        if col == len(b) + 1:
            col = 0
            row += 1
  
    return dis[len(a), len(b)]

# Данные

In [20]:
data = pd.read_csv('data.csv')
data = data.set_index('Unnamed: 0')

In [26]:
data.head()

,syllabic,stress,long,consonantal,sonorant,continuant,delayed release,approximant,tap,trill,...,anterior,distributed,strident,lateral,DORSAL,high,low,front,back,tense
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
ɒ,+,-,-,-,+,+,None,+,-,-,...,None,None,None,-,+,-,+,-,+,None
a,+,-,-,-,+,+,None,+,-,-,...,None,None,None,-,+,-,+,-,+,None
ɶ,+,-,-,-,+,+,None,+,-,-,...,None,None,None,-,+,-,+,+,-,None
ɑ,+,-,-,-,+,+,None,+,-,-,...,None,None,None,-,+,-,+,-,-,None
œ,+,-,-,-,+,+,None,+,-,-,...,None,None,None,-,+,-,+,+,-,None


# Non Language specific

## data

In [326]:
def sound_dist(a, b, data):
    
    '''
    реализовать stress = + при условии если ударный гласный
    ингнорирование лишней диакритики
    '''
    
    a = pd.get_dummies(data.loc[a]) 
    b = pd.get_dummies(data.loc[b])
    
    together = a + b
    
    similar = 0
    uncommon = 0
    common = 0

    if '+' in together and 2 in together['+'].value_counts():
        similar += together['+'].value_counts()[2]

    if '-' in together and 2 in together['-'].value_counts():
        similar += together['-'].value_counts()[2]

    if 'None' in together:
        None_values = together['None'].value_counts()
        
        if 1 in None_values:
            uncommon = None_values[1]

        if 0 in None_values:
            common = None_values[0]
        
    dist = 1 - (similar / (common + (uncommon * 2)))
    
    return dist


def non_ls_dist_matrix(data):
    
    col_ind = data.index.tolist()
    n_col = len(col_ind)
    dist_matrix = pd.DataFrame(np.zeros((n_col, n_col)),
                               index=col_ind, columns=col_ind)
    
    for i in itertools.combinations_with_replacement(data.index, 2):
            
            result = sound_dist(i[0], i[1], data)
            dist_matrix[i[0]][i[1]] = result
            dist_matrix[i[1]][i[0]] = result
    
    return dist_matrix

In [170]:
%%time
non_ls_dist = non_ls_dist_matrix(141, data)

CPU times: user 43.9 s, sys: 118 ms, total: 44 s
Wall time: 44.1 s


In [171]:
non_ls_dist.head()

,ɒ,a,ɶ,ɑ,œ,Ã,ɔ,o,ɤ,ɘ,...,ɰ,ɰ̠,w,ɥ,j,ɹ,ʋ,ʍ,ɦ,h
ɒ,0.000000,0.086957,0.086957,0.130435,0.173913,0.20,0.12,0.12,0.20,0.24,...,0.407407,0.28,0.20,0.28,0.36,0.575758,0.444444,0.407407,0.586207,0.620690
a,0.086957,0.000000,0.173913,0.043478,0.086957,0.12,0.20,0.20,0.12,0.16,...,0.333333,0.20,0.28,0.36,0.28,0.515152,0.444444,0.481481,0.517241,0.551724
ɶ,0.086957,0.173913,0.000000,0.130435,0.086957,0.28,0.20,0.20,0.28,0.24,...,0.407407,0.36,0.28,0.20,0.28,0.575758,0.444444,0.481481,0.586207,0.620690
ɑ,0.130435,0.043478,0.130435,0.000000,0.043478,0.16,0.24,0.24,0.16,0.12,...,0.333333,0.24,0.32,0.32,0.24,0.515152,0.444444,0.518519,0.517241,0.551724
œ,0.173913,0.086957,0.086957,0.043478,0.000000,0.20,0.28,0.28,0.20,0.16,...,0.333333,0.28,0.36,0.28,0.20,0.515152,0.444444,0.555556,0.517241,0.551724


# Language specific

## data

In [178]:
import csv

In [194]:
def open_data(path, delimiter):
    
    with open(path, 'r') as csvfile:
        
        reader = csv.reader(csvfile, delimiter=delimiter)
        data = list(reader)
    
    return data

In [334]:
words = open_data('test.csv',  ';')

In [482]:
reg = 'ɡ̠ɣ̠|t̠ɬ̠|k̟x̟|g̟ɣ̟|ŋ̟.1|d̠ɮ̠|k̠x̠|d͡ʑ|lò|kp|ɡb|pt|bd|dʒ|dz|dɮ|tʃ|ts|tɬ|ʈʂ|ɖʐ|pf|bv|pɸ|bβ|tθ|dð|cç|ɟʝ|kx|ɡɣ|qχ|ɢʀ|ts.1|tɬ.1|dz.1|dɮ.1|œ.1|ʟ̠|ŋ̟|ʟ̟|ɣ̟|x̟|k̟|ɡ̟|ɣ̠|x̠|k̠|ɡ̠|ɰ̠|ɒ|a|ɶ|ɑ|œ|Ã|ɔ|o|ɤ|ɘ|ə|e|ɞ|ø|ɛ|ɵ|ɯ|u|ʊ|ɨ|ʉ|y|i|ʏ|ɪ|ɴ|ʀ|ɲ|ʎ|ŋ|ʟ|ɳ|ʙ|ɭ|ä|ɻ|ɽ|r|n|m|l|ɾ|ɱ|ʔ|ħ|ʕ|ʁ|q|χ|ɢ|ɕ|ɟ|ʝ|c|ç|tɕ|ɣ|x|k|ɡ|ʑ|ʈ|ɖ|ɬ|ʐ|ɸ|ʂ|ʒ|z|v|t|ʃ|s|p|f|d|b|θ|ɮ|ð|β|ɧ|ɰ|w|ɥ|j|ɹ|ʋ|ʍ|ɦ|h'

# Делим транскрипцию на части

In [490]:
def phone_splitter(word):
    
    '''
        + - advanced
        _ - retracted
        tʲ - мягкий (back +)
        tʷ - Labalized (round + )
        dⁿ - nasal +
        _1 - stress + 
        tˠ - Velarized
        t̬ - voiced
        d̥ - voiceless
    '''
    
    
    def find_stress_vow(word):
        
        '''
        Ищет ударные гласный и дифтонги
        '''
        r_stress = '_[1-4]'
        f = re.findall(r_stress, word)
        st_reg = ''
        
        for i in f:
            st_reg += ('.'*int(i[-1])) + i + '|'
        return st_reg
    
    
    word = re.sub('͡', '_аффрик_', word)
    word = re.sub('̥', '_глух', word)
    word = re.sub('̬', '_звонк', word)
    r_stress = find_stress_vow(word)    # _1

    all_ = '(' + r_stress + '._аффрик_.' + '|' + '._звонк' + '|' + '._глух' + '|' + '.ⁿ' + '|' + '.ʲ' + '|' + '.ʷ' + '|' + reg + '|' + '.'+ ')'
    res = re.findall(all_, word)

    return res + ['<eos>']

In [491]:
word = 'pˠraɒ_2td͡ʑək̟tʟ̠tʷdⁿtʲtˠd̥t̬ʧdʒtʃ'
phone_splitter(word)

['p',
 'ˠ',
 'r',
 'aɒ_2',
 't',
 'd_аффрик_ʑ',
 'ə',
 'k̟',
 't',
 'ʟ̠',
 'tʷ',
 'dⁿ',
 'tʲ',
 't',
 'ˠ',
 'd_глух',
 't_звонк',
 'ʧ',
 'dʒ',
 'tʃ',
 '<eos>']

In [492]:
for i in words:
    for l in i:
        print(phone_splitter(l))

['m', 'e', 'i', 't', '<eos>']
['h', 'e', 'i', 't', '<eos>']
['l', 'a', 'v', '<eos>']
['p', 'r', 'ə', 'k', 't', '<eos>']


In [493]:
def all_phones(arr):

    phones = []

    for line in arr:

        if len(line) != 2:
            raise ValueError('There have to be 2 columns in a line. Check your delimiter.')

        for word in line:

            if word == '':
                raise ValueError('All columns have to be filled.')
            phones += phone_splitter(word)

        phones.append('<\n>')

    return phones

In [494]:
all_phones(words)

['m',
 'e',
 'i',
 't',
 '<eos>',
 'h',
 'e',
 'i',
 't',
 '<eos>',
 '<\n>',
 'l',
 'a',
 'v',
 '<eos>',
 'p',
 'r',
 'ə',
 'k',
 't',
 '<eos>',
 '<\n>']

# irrelevant_features

In [275]:
def irrelevant_features(phones, data):
    
    '''
    Надо ли удалять звуки, которые не встретились?
    '''
    
    features = set()
    
    for index, phone in enumerate(set(phones)):
        
        if phone not in ('<\n>', '<eos>'):
        
            res = set(data.loc[phone].loc[data.loc[phone] == 'None'].keys())
            
            if index == 0: features = res
            else: features &= res

    return features

In [330]:
def ls_dist_matrix(phones, data):
    
    irr_f = irrelevant_features(phones, data)
    data = data.drop(columns=irr_f)
    
    dist_matrix = non_ls_dist_matrix(data)
    
    return dist_matrix

In [331]:
ls_dist = ls_dist_matrix(phones, data)
ls_dist.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in long_scalars


,ɒ,a,ɶ,ɑ,œ,Ã,ɔ,o,ɤ,ɘ,...,ɰ,ɰ̠,w,ɥ,j,ɹ,ʋ,ʍ,ɦ,h
ɒ,0.000000,0.105263,0.000000,0.105263,0.105263,0.105263,0.000000,0.000000,0.105263,0.105263,...,0.157895,0.157895,0.052632,0.052632,0.157895,0.44,0.210526,0.333333,0.428571,0.476190
a,0.105263,0.000000,0.105263,0.000000,0.000000,0.000000,0.105263,0.105263,0.000000,0.000000,...,0.052632,0.052632,0.157895,0.157895,0.052632,0.36,0.210526,0.428571,0.333333,0.380952
ɶ,0.000000,0.105263,0.000000,0.105263,0.105263,0.105263,0.000000,0.000000,0.105263,0.105263,...,0.157895,0.157895,0.052632,0.052632,0.157895,0.44,0.210526,0.333333,0.428571,0.476190
ɑ,0.105263,0.000000,0.105263,0.000000,0.000000,0.000000,0.105263,0.105263,0.000000,0.000000,...,0.052632,0.052632,0.157895,0.157895,0.052632,0.36,0.210526,0.428571,0.333333,0.380952
œ,0.105263,0.000000,0.105263,0.000000,0.000000,0.000000,0.105263,0.105263,0.000000,0.000000,...,0.052632,0.052632,0.157895,0.157895,0.052632,0.36,0.210526,0.428571,0.333333,0.380952


# all together

In [333]:
class PhoneticAlgorithmIPA:
    
    def __init__(self, data_path, data_delimiter, problem_type, ft_path='', feature_delimiter=';'):
        
        
        self.phones = self.data_splitter(data_path, data_delimiter)
        self.feature_table = self.open_ftable(ft_path, feature_delimiter)
        
        if problem_type == 'LS':
            self.dist_matrix = ls_dist_matrix(phones, data)
        if problem_type == 'Non_LS':
            self.dist_matrix = non_ls_dist_matrix(data) 
        else: raise ValueError('Problem type can be either LS or Non_LS.')
            
           
        
        
        
        
        def open_data(self, path, delimiter):
            with open(path, 'r') as csvfile:
                reader = csv.reader(csvfile, delimiter=delimiter)
                data = list(reader)
            return data
        
        
        
        
        
        
        def data_splitter():
            return
        
        
        
        
        
        
        def open_ftable(ft_path, delimiter):
            return
        
        
        
        
        def rebuild_dist_matrix():
            return
        
        
        
        
        def phone_splitter(word):
    
            '''
                + - advanced
                _ - retracted
                tʲ - мягкий (back +)
                tʷ - Labalized (round + )
                dⁿ - nasal +
                _1 - stress + 
                tˠ - Velarized
                t̬ - voiced
                d̥ - voiceless
            '''
    
    
            def find_stress_vow(word):
        
                '''
                Ищет ударные гласный и дифтонги
                '''
                r_stress = '_[1-4]'
                f = re.findall(r_stress, word)
                st_reg = ''
        
                for i in f:
                    st_reg += ('.'*int(i[-1])) + i + '|'
                return st_reg
    
    
            word = re.sub('͡', '_аффрик_', word)
            word = re.sub('̥', '_глух', word)
            word = re.sub('̬', '_звонк', word)
            r_stress = find_stress_vow(word)    # _1

            all_ = '(' + r_stress + '._аффрик_.' + '|' + '._звонк' + '|' + '._глух' 
            all_ += '|' + '.ⁿ' + '|' + '.ʲ' + '|' + '.ʷ' + '|' + reg + '|' + '.'+ ')'
            res = re.findall(all_, word)

            return res
        
        
        
        
        def sound_dist(a, b, data):
            a = pd.get_dummies(data.loc[a]) 
            b = pd.get_dummies(data.loc[b])

            together = a + b
            similar = 0
            uncommon = 0
            common = 0

            if '+' in together and 2 in together['+'].value_counts():
                similar += together['+'].value_counts()[2]
            if '-' in together and 2 in together['-'].value_counts():
                similar += together['-'].value_counts()[2]
            if 'None' in together:
                None_values = together['None'].value_counts()
            if 1 in None_values:
                uncommon = None_values[1]
            if 0 in None_values:
                common = None_values[0]
            dist = 1 - (similar / (common + (uncommon * 2)))
            return dist

        
        
        

        def non_ls_dist_matrix(data):
            col_ind = data.index.tolist()
            n_col = len(col_ind)
            dist_matrix = pd.DataFrame(np.zeros((n_col, n_col)),
                               index=col_ind, columns=col_ind)
            for i in itertools.combinations_with_replacement(data.index, 2):
                result = sound_dist(i[0], i[1], data)
                dist_matrix[i[0]][i[1]] = result
                dist_matrix[i[1]][i[0]] = result
            return dist_matrix
        
        
            
            
            
        def irrelevant_features(phones, data):
            '''
            Надо ли удалять звуки, которые не встретились?
            '''
            features = set()
            for index, phone in enumerate(set(phones)):
                if phone not in ('<\n>', '<eos>'):
                    res = set(data.loc[phone].loc[data.loc[phone] == 'None'].keys())
                    if index == 0: features = res
                    else: features &= res
            return features
        
        
        
        
        def ls_dist_matrix(phones, data):
            irr_f = irrelevant_features(phones, data)
            data = data.drop(columns=irr_f)
            dist_matrix = non_ls_dist_matrix(data)
            return dist_matrix 
        

In [ ]:
добавить читаемость символов

n̥ d̥	805	0325	voiceless
ŋ̊	778	030A	voiceless (use if character has descender)
b̤ a̤	804	0324	breathy voiced
t̪ d̪	810	032A	dental
s̬ t̬	812	032C	voiced